In [1]:
# Dependencies
import requests
import json
import pandas as pd
import gmaps
import time

# Google developer API key
from config import gkey
gmaps.configure(gkey)

In [2]:
# import csv file into a DataFrame
weather_data = pd.read_csv("Weather_Output/Weather_DataFrame.csv")

In [3]:
# create a heat map of humidity per city
locations = weather_data[["Latitude","Longitude"]]
humidity = weather_data["Humidity (%)"]

figure_layout = {"width":"900px",
                 "height":"500px",
                 "border":"1px solic black",
                 "padding":"1px","margin":"0 auto 0 auto"}
fig = gmaps.figure(layout=figure_layout,center=(0, 0), zoom_level=1.5,map_type="HYBRID")
humidity_layer = gmaps.heatmap_layer(locations,weights=humidity,dissipating=False,point_radius=1,
                                    max_intensity = max(weather_data['Humidity (%)']))
max_intensity = max(weather_data['Humidity (%)'])
fig.add_layer(humidity_layer)
fig

Figure(layout=FigureLayout(border='1px solic black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

In [4]:
# filter the DataFrame based on personal preferences of temp(50-80F), cloudiness(<20%), and windspeed(<10mph)
cond = (weather_data['Temperature (F)'].isin(range(50,80)))&(
        weather_data['Cloudiness (%)']<20)&(
        weather_data['Wind Speed (mph)']<10)
vacation_spots = weather_data.iloc[weather_data[cond].index]
vacation_spots

,City,Country,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Hemisphere
7,saldanha,za,-33.01,17.94,50.0,81,0,3.36,Southern
15,mount isa,au,-20.73,139.50,59.0,41,0,5.82,Southern
335,presidencia roque saenz pena,ar,-26.79,-60.44,73.0,69,0,6.85,Southern
598,aykhal,ru,66.00,111.50,50.0,61,0,8.95,Northern
887,behbahan,ir,30.60,50.24,77.0,17,0,9.10,Northern


In [5]:
# use google places api to find first hotel within 5000 meters of coordinates
params={"radius": 5000, "type":"hotel" , "key":gkey}

# loop though the vacation spots DataFrame to find hotel
for index,row in vacation_spots.iterrows():
    lat = row['Latitude']
    lng = row['Longitude']
    
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotels = requests.get(base_url,params=params)
    hotels = hotels.json()
    try:
        vacation_spots.loc[index,"Hotels"] = hotels["results"][0]["name"]
    except:
        print("Missing field/result... skiping")

C:\Users\DINKS\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\DINKS\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [6]:
vacation_spots

,City,Country,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Hemisphere,Hotels
7,saldanha,za,-33.01,17.94,50.0,81,0,3.36,Southern,Saldanha
15,mount isa,au,-20.73,139.50,59.0,41,0,5.82,Southern,Mount Isa
335,presidencia roque saenz pena,ar,-26.79,-60.44,73.0,69,0,6.85,Southern,Presidencia Roque Sáenz Peña
598,aykhal,ru,66.00,111.50,50.0,61,0,8.95,Northern,Novoye Kladbishche
887,behbahan,ir,30.60,50.24,77.0,17,0,9.10,Northern,Behbahan


In [7]:
# plot the hotel layer on top of the humidity heat map
city_location = vacation_spots[["Latitude","Longitude"]]
hotels = vacation_spots["Hotels"].tolist()
city = vacation_spots["City"].tolist()
country = vacation_spots["Country"].tolist()
info_lst = list(zip(hotels,city,country))
city_markers = gmaps.marker_layer(city_location,
                                 info_box_content=[f"Hotel: {info}" for info in info_lst])
fig.add_layer(city_markers)
fig

Figure(layout=FigureLayout(border='1px solic black', height='500px', margin='0 auto 0 auto', padding='1px', wi…